In [1]:
#%pip install --upgrade pip

In [1]:
%load_ext autoreload
%autoreload 2
%pip list | grep cv


cvpartnerpy        0.0.2      /Users/eirik.stavelin/Projects/cvpartnerpy
Note: you may need to restart the kernel to use updated packages.


In [82]:

import os
import pandas as pd
from cvpartner import CVPartner
from cvpartner.helpers import get_new_certification, newest_project_is_older_than_n_months, sort_projects_newest_to_oldest


from dotenv import load_dotenv
load_dotenv()

False

In [72]:

# når var deres eksamensår?
cvp = CVPartner(org='noaignite', api_key=os.environ['CVPARTNER_API_KEY'])
cvp.list_offices()  # 'Frontend', 'Platforms & Services'


[('56d7f59569702d248e000004', 'Admin'),
 ('5fd0b63c7958da0e882f6e8e', 'CX'),
 ('56d80bf269702d41da00000a', 'Client Ops'),
 ('56d80bde69702d41da000008', 'Content Design'),
 ('63b3d70930107c100a2bba48', 'Customer Success Services'),
 ('56d7f60a69702d248e000008', 'Data Engineering'),
 ('63b3d4cb40717e0fb2e8b95a', 'Executive Advisory'),
 ('56d7f5fd69702d248e000007', 'Frontend'),
 ('56d7f5e169702d248e000005', 'Platforms & Services'),
 ('63b3d477eef8220fd65d7847', 'Product Management'),
 ('56d80bec69702d41da000009', 'Project Management'),
 ('5fd0b8067958da0e892f6efd', 'UX')]

In [75]:
# Fast but, only users, not cvs
dataeng = cvp.get_department_by_name(office_name='Data Engineering')

for person in dataeng:
    print(person.get('name'))
    #print(person.keys())
len(dataeng)

Anders Haukvik
Anita Nicole Lipsky
Bendik Austnes
Christoffer Holm-Kjøhl
Dragos-Luchian Ismana
Eirik Stavelin
Emanuele Caprioli
Erik Martinsen
Erlend Øien
Giske Naper Freberg
Hans Erik Lange
Håkon Kvalen
Inger Skjærholt
Iselin Eng
Jan Tore Kyrdalen
Johan Cheu Hang Hoi
Johanne Skogvang
Julian Andersen
Jørgen Hellemann
Karina Borlaug
Lars Christian Kolsaker
Marianne Pettersen
Nikolai Molvik
Paulina Zientarska
Peder Frang
Robert Kolner
Tushar Thakkar
Victor Undli
Örjan Andersson
Øyvind Bratvedt


30

In [76]:
# Fast, 8.2s for these three (only users)
#backend = cvp.get_department_by_name('Platforms & Services')
#frontend = cvp.get_department_by_name('Frontend')
#data_engineering = cvp.get_department_by_name()


In [77]:

# ah.. also slow. 1m50s for these three, including CVs
#backend = cvp.get_users_and_cvs_from_department('Platforms & Services')
#frontend = cvp.get_users_and_cvs_from_department('Frontend')
#data_engineering = cvp.get_users_and_cvs_from_department()


In [78]:
data_engineering = cvp.get_users_and_cvs_from_department()


In [59]:
# slow 1m50s for these three
#backend = cvp.get_department('Platforms & Services')
#frontend = cvp.get_department('Frontend')
#data_engineering = cvp.get_department()


In [79]:
dept = data_engineering

In [81]:
new_certifications: list[tuple[str, list[dict]]] = []

print("Looking for new certifications...")
for person, cv in dept[:]:
    new_certs = get_new_certification(cv, 
                                      days_to_look_back=1000)
    if new_certs:
        new_certifications.append(
            (cv.get('navn'), new_certs)
        )
print(len(new_certifications), "new certifications found")
for name, certs in new_certifications:
    print(name)
    for cert in certs:
        print(f"\t- {cert.get('name').get('no')}")


Victor Undli har en uten årstall: ISTQB Foundation Certificate in Software testing


Looking for new certifications...
11 new certifications found
Anita Nicole Lipsky
	- Elastic Certified Analyst
	- Azure Data Fundamentals (DP-900)
	- Elastic Certified Observability Engineer
Bendik Austnes
	- Databricks Lakehouse Fundamentals Accreditaiton
Eirik Stavelin
	- PRINCE2 Foundation
Erik Martinsen
	- Episerver Certified Developer
Giske Naper Freberg
	- DP-900: Azure Data Fundamentals
Jan Tore Kyrdalen
	- Certified Scrum Master
	- Elastic Certified Engineer
	- Terraform Associate (002)
Nikolai Molvik
	- Databricks Certified Associate Developer for Apache Spark 3.0
Peder Frang
	- DP-900: Azure Data Fundamentals
	- Azure Data Engineer Associate
	- Developer Foundations
Tushar Thakkar
	- Professional Scrum Master
	- DP-900: Azure Data Fundamentals
Örjan Andersson
	- Optimizely Content Cloud Developer Certification
Øyvind Bratvedt
	- Azure Data Engineer Associate


In [83]:
# find CVs that lacks updated prosjetc
people_who_needs_to_update_cv: list[tuple[str, dict]] = []

for person, cv in dept[:30]:
    print()
    print(f"{cv.get('navn')} newest_project_is_older_than_n_months: {newest_project_is_older_than_n_months(cv)}")

    sorted_projects = sort_projects_newest_to_oldest(cv)
    # print(sorted_projects)
    for date_from, date_to, delta_monts, project in sorted_projects:
        print(
            f"\t fra: {date_from.date()} - til: {date_to.date() if date_to else None} {delta_monts}mnd @ {project.get('customer').get('no')}")






Anders Haukvik newest_project_is_older_than_n_months: False
	 fra: 2020-01-01 - til: None 39mnd @ Studentsamskipnaden i Oslo (SiO)
	 fra: 2018-08-01 - til: 2019-03-01 7mnd @ Standard Norge
	 fra: 2017-01-01 - til: 2019-10-01 33mnd @ Standard Norge
	 fra: 2015-11-01 - til: 2016-12-01 13mnd @ NorgesGruppen
	 fra: 2015-08-01 - til: 2015-10-01 2mnd @ Responsive Norge
	 fra: 2014-05-01 - til: 2014-12-01 7mnd @ TradeSolution
	 fra: 2013-01-01 - til: 2015-08-01 31mnd @ TradeSolution
	 fra: 2012-08-01 - til: 2012-12-01 4mnd @ Statens Kartverk
	 fra: 2012-05-01 - til: 2012-08-01 3mnd @ Statens Helsetilsyn
	 fra: 2010-02-01 - til: 2012-01-01 23mnd @ Norsk Rikstoto
	 fra: 2007-11-01 - til: 2009-12-01 25mnd @ Norsk Rikstoto
	 fra: 2007-08-01 - til: 2007-11-01 3mnd @ Norsk Rikstoto
	 fra: 2007-05-01 - til: 2007-10-01 5mnd @ Norsk Rikstoto
	 fra: 2007-01-01 - til: 2007-04-01 3mnd @ Norsk Rikstoto
	 fra: 2006-11-01 - til: 2007-02-01 3mnd @ Rikstoto
	 fra: 2006-03-01 - til: 2006-08-01 5mnd @ Statens 